# Importações

In [1]:
import pandas as pd 
import numpy as np
import re
from utilsforecast.preprocessing import fill_gaps

# Importar dados do PostgreSQL

In [2]:
# Como a Api ainda não esta a funcionar, vou importar os dados de um csv
df = pd.read_csv('../all_bike_counts.csv', parse_dates=['detected'], dtype='int32')
df = df.set_index('detected')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22632 entries, 2021-08-01 00:00:00 to 2024-02-29 23:00:00
Data columns (total 62 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   direction_2   22632 non-null  int32
 1   count_2       22632 non-null  int32
 2   direction_3   22632 non-null  int32
 3   count_3       22632 non-null  int32
 4   direction_4   22632 non-null  int32
 5   count_4       22632 non-null  int32
 6   direction_5   22632 non-null  int32
 7   count_5       22632 non-null  int32
 8   direction_6   22632 non-null  int32
 9   count_6       22632 non-null  int32
 10  direction_7   22632 non-null  int32
 11  count_7       22632 non-null  int32
 12  direction_8   22632 non-null  int32
 13  count_8       22632 non-null  int32
 14  direction_9   22632 non-null  int32
 15  count_9       22632 non-null  int32
 16  direction_10  22632 non-null  int32
 17  count_10      22632 non-null  int32
 18  direction_11  22632 non-null  i

In [3]:
df_loc = pd.read_csv('../all_counter_locations.csv')
df_loc.info()
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   locationId         31 non-null     int64  
 1   tenantIdentifier   31 non-null     object 
 2   name               31 non-null     object 
 3   cp7                31 non-null     object 
 4   freguesia          31 non-null     object 
 5   directionPositive  0 non-null      float64
 6   directionNegative  0 non-null      float64
 7   latitude           31 non-null     float64
 8   longitude          31 non-null     float64
dtypes: float64(4), int64(1), object(4)
memory usage: 2.3+ KB



# Analises

In [4]:
df.index

DatetimeIndex(['2021-08-01 00:00:00', '2021-08-01 01:00:00',
               '2021-08-01 02:00:00', '2021-08-01 03:00:00',
               '2021-08-01 04:00:00', '2021-08-01 05:00:00',
               '2021-08-01 06:00:00', '2021-08-01 07:00:00',
               '2021-08-01 08:00:00', '2021-08-01 09:00:00',
               ...
               '2024-02-29 14:00:00', '2024-02-29 15:00:00',
               '2024-02-29 16:00:00', '2024-02-29 17:00:00',
               '2024-02-29 18:00:00', '2024-02-29 19:00:00',
               '2024-02-29 20:00:00', '2024-02-29 21:00:00',
               '2024-02-29 22:00:00', '2024-02-29 23:00:00'],
              dtype='datetime64[ns]', name='detected', length=22632, freq=None)

In [5]:
df.loc['2022'].tail(2)

,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,count_6,...,direction_31,count_31,direction_32,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35
detected,,,,,,,,,,,,,,,,,,,,,
2022-12-31 22:00:00,0,7,2,2,2,2,2,5,9,15,...,1,1,1,15,26,127,52,87,12,55
2022-12-31 23:00:00,1,11,0,1,1,3,5,6,10,14,...,2,2,4,14,15,238,99,138,24,100


In [6]:
df.resample('D').first().dropna().index[[0, -1]]

DatetimeIndex(['2021-08-01', '2024-02-29'], dtype='datetime64[ns]', name='detected', freq=None)

In [7]:
df.resample('D').first().dropna().iloc[[0, -1]]

,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,count_6,...,direction_31,count_31,direction_32,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35
detected,,,,,,,,,,,,,,,,,,,,,
2021-08-01,0,0,1,1,0,0,0,0,6,7,...,1,1,3,7,4,6,2,2,30,42
2024-02-29,6,7,0,1,4,6,1,7,3,12,...,1,3,2,5,8,11,2,13,5,17


In [8]:
# Seleciona o primeiro e o último dia com os contadores
df_mostrar = df.resample('D').first().dropna().iloc[[0, -1]]

# Identifica as colunas que representam os contadores e direções
contador_cols = [col for col in df.columns if col.startswith('direction_') or col.startswith('count_')]

# Identifica o primeiro e o último contador
primeiro_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[0]
ultimo_contador = sorted(set(int(col.split('_')[1]) for col in contador_cols))[-1]

# Seleciona as colunas para o primeiro e o último contador
primeiro_contador_cols = [f'direction_{primeiro_contador}', f'count_{primeiro_contador}']
ultimo_contador_cols = [f'direction_{ultimo_contador}', f'count_{ultimo_contador}']

# Exibe as colunas desejadas
df_selecionado = df_mostrar[primeiro_contador_cols + ultimo_contador_cols]

# Exibe o resultado
df_selecionado


,direction_2,count_2,direction_35,count_35
detected,,,,
2021-08-01,0,0,30,42
2024-02-29,6,7,5,17


In [9]:
df_loc.head(2)

,locationId,tenantIdentifier,name,cp7,freguesia,directionPositive,directionNegative,latitude,longitude
0,2,lisboa.pt.mycycletraffic,#18 - Avenida do Colégio Militar,1500-069,Benfica,NaN,NaN,38.749623,-9.191153
1,3,lisboa.pt.mycycletraffic,#01 - Telheiras - Colégio Alemão,1600-616,Lumiar,NaN,NaN,38.759682,-9.164044


### Identificar se há dias com valores a zéro

In [10]:
df_daily = df.resample('D').sum()

In [11]:
import re

def analisar_zeros(df_daily, contadores, df_loc):
    resultados = []

    for contador in contadores:
        if contador in df_daily.columns:
            series = df_daily[contador].copy()
            zero_days = series[series == 0]

            if not zero_days.empty:
                match = re.match(r'count_(\d+)', contador)
                if match:
                    location_id = int(match.group(1))
                    try:
                        location_name = df_loc.query("locationId == @location_id").name.values[0]
                    except IndexError:
                        location_name = "Nome desconhecido"

                    diffs = zero_days.index.to_series().diff().dt.days.fillna(1)
                    grupo = (diffs != 1).cumsum()
                    sequencias = zero_days.groupby(grupo)

                    detalhes_sequencias = []
                    for _, seq in sequencias:
                        detalhes_sequencias.append({
                            'inicio': seq.index[0].date(),
                            'fim': seq.index[-1].date(),
                            'dias': len(seq)
                        })

                    resultados.append({
                        'contador': contador,
                        'nome': location_name,
                        'total_dias_zeros': len(zero_days),
                        'n_sequencias': len(detalhes_sequencias),
                        'sequencias': detalhes_sequencias
                    })

    return resultados




In [12]:
contadores = [col for col in df_daily.columns if col.startswith('count_')]
resultados_zeros = analisar_zeros(df_daily, contadores, df_loc)

for r in resultados_zeros:
    print(f"\nContador: {r['contador']} ({r['nome']})")
    print(f"Total de dias com zeros: {r['total_dias_zeros']}")
    print(f"Número de sequências: {r['n_sequencias']}")
    for seq in r['sequencias']:
        print(f" - De {seq['inicio']} a {seq['fim']} ({seq['dias']} dias)")



Contador: count_2 (#18 - Avenida do Colégio Militar)
Total de dias com zeros: 14
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_3 (#01 - Telheiras - Colégio Alemão)
Total de dias com zeros: 20
Número de sequências: 5
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2023-12-01 a 2023-12-04 (4 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_4 (#02 - Avenida do Brasil - Mata de Alvalade)
Total de dias com zeros: 16
Número de sequências: 4
 - De 2022-08-18 a 2022-08-28 (11 dias)
 - De 2023-01-20 a 2023-01-20 (1 dias)
 - De 2023-10-20 a 2023-10-22 (3 dias)
 - De 2024-01-19 a 2024-01-19 (1 dias)

Contador: count_5 (#03 - Avenida do Brasil - Campo Grande)
Total de dias com zeros: 14
Número de sequências: 4
 - De 2022-08-20 a 2022-08-28 (9 dias)
 - De 2023-0

### Fill gaps 

In [21]:
df = df.reset_index()

df.rename(columns={'detected': 'ds'}, inplace=True)

df['unique_id'] = 1

# 4. Criar coluna auxiliar com a data (sem hora)
df['datetime'] = df['ds'].dt.date

In [22]:
df

,ds,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,...,direction_32,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35,unique_id,datetime
0,2021-08-01 00:00:00,0,0,1,1,0,0,0,0,6,...,3,7,4,6,2,2,30,42,1,2021-08-01
1,2021-08-01 01:00:00,3,3,0,2,0,0,0,0,0,...,1,7,7,7,2,7,13,21,1,2021-08-01
2,2021-08-01 02:00:00,0,0,1,1,0,2,0,0,0,...,0,0,0,2,3,5,13,19,1,2021-08-01
3,2021-08-01 03:00:00,0,0,0,0,0,0,0,0,2,...,0,0,0,1,0,1,8,11,1,2021-08-01
4,2021-08-01 04:00:00,0,0,0,0,0,3,1,1,0,...,1,1,0,1,2,3,4,8,1,2021-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,2024-02-29 19:00:00,15,35,26,240,13,32,22,77,30,...,3,21,17,47,19,81,13,43,1,2024-02-29
22628,2024-02-29 20:00:00,17,35,11,144,11,17,20,46,12,...,3,9,17,29,13,49,14,39,1,2024-02-29
22629,2024-02-29 21:00:00,11,13,2,22,10,13,7,19,6,...,3,10,8,13,17,42,14,46,1,2024-02-29
22630,2024-02-29 22:00:00,6,20,174,177,4,5,14,21,18,...,2,2,5,12,11,31,8,24,1,2024-02-29


In [23]:

# Identificar todas as colunas de contagem (que começam com 'count_')
count_cols = [col for col in df.columns if col.startswith('count_')]

# Agora vamos substituir os dias inteiros com zeros por NaN
for col in count_cols:
    # Identificar os dias onde todos os valores de 'count_*' são zero
    dias_zeros = df.groupby('datetime')[col].apply(lambda x: (x == 0).all())
    dias_zeros = dias_zeros[dias_zeros].index  # filtra só os dias que têm todos os contadores em zero

    # Substituir os valores desses dias por NaN
    df.loc[df['datetime'].isin(dias_zeros), col] = pd.NA

# Verificar se os NaNs foram aplicados corretamente
print(df.head())

                   ds  direction_2  count_2  direction_3  count_3  \
0 2021-08-01 00:00:00            0      0.0            1      1.0   
1 2021-08-01 01:00:00            3      3.0            0      2.0   
2 2021-08-01 02:00:00            0      0.0            1      1.0   
3 2021-08-01 03:00:00            0      0.0            0      0.0   
4 2021-08-01 04:00:00            0      0.0            0      0.0   

   direction_4  count_4  direction_5  count_5  direction_6  ...  direction_32  \
0            0      0.0            0      0.0            6  ...             3   
1            0      0.0            0      0.0            0  ...             1   
2            0      2.0            0      0.0            0  ...             0   
3            0      0.0            0      0.0            2  ...             0   
4            0      3.0            1      1.0            0  ...             1   

   count_32  direction_33  count_33  direction_34  count_34  direction_35  \
0       7.0          

In [24]:
# Verifique os NaNs para garantir que só os dias inteiros com zero foram substituídos
print(df.isna().sum())  # Veja a quantidade de NaNs em cada coluna

# Verifique os dias que foram alterados
dias_alterados = df[df.isna().any(axis=1)]['datetime'].unique()
print(f"Dias com NaN: {dias_alterados}")


ds                0
direction_2       0
count_2         336
direction_3       0
count_3         480
               ... 
count_34        552
direction_35      0
count_35        336
unique_id         0
datetime          0
Length: 65, dtype: int64
Dias com NaN: [datetime.date(2021, 10, 2) datetime.date(2021, 10, 3)
 datetime.date(2022, 5, 21) datetime.date(2022, 5, 22)
 datetime.date(2022, 8, 18) datetime.date(2022, 8, 19)
 datetime.date(2022, 8, 20) datetime.date(2022, 8, 21)
 datetime.date(2022, 8, 22) datetime.date(2022, 8, 23)
 datetime.date(2022, 8, 24) datetime.date(2022, 8, 25)
 datetime.date(2022, 8, 26) datetime.date(2022, 8, 27)
 datetime.date(2022, 8, 28) datetime.date(2022, 10, 2)
 datetime.date(2022, 11, 26) datetime.date(2022, 12, 10)
 datetime.date(2022, 12, 11) datetime.date(2022, 12, 17)
 datetime.date(2022, 12, 18) datetime.date(2023, 1, 16)
 datetime.date(2023, 1, 20) datetime.date(2023, 6, 3)
 datetime.date(2023, 7, 30) datetime.date(2023, 10, 20)
 datetime.date(2023, 

In [25]:
# Preencher os gaps com a frequência desejada, por exemplo, hora ('H')
df_filled = fill_gaps(df, freq='H', id_col='unique_id', time_col='ds')

# Verificar se o preenchimento foi feito corretamente
print(f"Número de linhas após preencher os gaps: {len(df_filled)}")




Número de linhas após preencher os gaps: 22632


c:\Users\joana\AppData\Local\Programs\Python\Python312\Lib\site-packages\utilsforecast\preprocessing.py:131: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = pd.tseries.frequencies.to_offset(freq)


In [26]:
# Contar o número de NaNs por coluna
print(df.isna().sum())


ds                0
direction_2       0
count_2         336
direction_3       0
count_3         480
               ... 
count_34        552
direction_35      0
count_35        336
unique_id         0
datetime          0
Length: 65, dtype: int64
